In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import os
import random
import seaborn as sns

### Generate data and intsert into database

list of vars: (total of 10)
patient_id, similarity, roi_dose, ptv_dose, ptv_volume, roi_roi_distance,
roi_roi_overlap, rt_dose, roi, institution

In [2]:
# generate data (600 patients, 100 ucla, 200 suny b, 300 others)
# 1. patient ID, from UCLA/SUNY_B
patient_id = []
for i in list(range(1,101)):
    inst_name = 'UCLA_PR_'
    pid = inst_name+str(i)
    patient_id.append(pid)
for i in list(range(1,201)):
    inst_name = 'SUNY_B_PR_'
    pid = inst_name+str(i)
    patient_id.append(pid)
for i in list(range(1,301)):
    inst_name = 'OTHERS_PR_'
    pid = inst_name+str(i)
    patient_id.append(pid)
print(len(patient_id))

600


In [3]:
# # 2. similarity 0~0.99
# similarity = list(np.random.uniform(0.1,0.99,600).round(2))

# 3. PTV dose 0~100 Gy
ptv_dose = list(np.random.normal(40,5,600).round(2))
# print("PTV dose", ptv_dose[:10])

# 4. roi dose 0~100 Gy (ptv_dose - a proportion of itself)
roi_dose = [round(i- i*(np.random.normal(0.5,0.2)),2) for i in ptv_dose]
# print("ROI dose", roi_dose[:10])

# 5. PTV volume 0~194 mm^3
ptv_volume = list(np.random.normal(130,30,600).round(2))

# 6. ROI-ROI distance 0~99mm
roi_roi_distance = list(np.random.normal(50,20,600).round(2))

# 7. ROI-ROI overlap 0~99%
roi_roi_overlap = list(np.random.normal(0.5,0.3,600).round(2))

# 8. RT dose 40~60 Gy
rt_dose = list(np.random.uniform(50,10,600).round(2))

# 9. roi 
roi_possible_list = ['Brainstem', 'Cochlea R', 'Cochlea L', 'Dental Amalgam', 'GTV','Lacrimal R','Lens R',
                    'Optic Chiasm','Optic Nerve','PTV','Retina R','Teeth']
np.random.seed(22)
roi = list(np.random.choice(a = roi_possible_list, size=600))

In [4]:
# create dataframe (deleted similarity)
col_in_db0 = ['patient_id','roi_dose','ptv_dose','ptv_volume',\
             'roi_roi_distance','roi_roi_overlap','rt_dose',\
            'roi']
df = pd.DataFrame(list(zip(patient_id, roi_dose, ptv_dose,\
                          ptv_volume, roi_roi_distance, roi_roi_overlap, rt_dose,\
                          roi)), columns = col_in_db0)

In [5]:
# 10. institution: generate later
df['pos'] = df['patient_id'].str.find('_PR_')
df['institution'] = df.apply(lambda x:x['patient_id'][0:x['pos']], axis=1)
df.drop(columns=['pos'],inplace=True)
df.head(10)

,patient_id,roi_dose,ptv_dose,ptv_volume,roi_roi_distance,roi_roi_overlap,rt_dose,roi,institution
0,UCLA_PR_1,2.40,39.55,133.52,70.14,0.52,25.40,Lacrimal R,UCLA
1,UCLA_PR_2,20.38,31.76,83.78,72.53,0.72,35.82,GTV,UCLA
2,UCLA_PR_3,9.29,34.85,144.42,52.20,0.57,39.99,Brainstem,UCLA
3,UCLA_PR_4,25.62,35.00,107.16,77.22,-0.02,22.12,GTV,UCLA
4,UCLA_PR_5,21.22,35.43,130.13,36.90,0.51,12.89,Lens R,UCLA
5,UCLA_PR_6,16.79,35.95,112.88,49.95,0.73,23.01,Lens R,UCLA
6,UCLA_PR_7,8.95,43.94,91.55,39.11,0.47,20.74,Teeth,UCLA
7,UCLA_PR_8,23.61,38.35,65.08,73.42,0.82,39.95,GTV,UCLA
8,UCLA_PR_9,9.23,28.83,171.49,49.49,0.27,47.56,Optic Nerve,UCLA
9,UCLA_PR_10,19.41,42.59,87.32,67.45,0.41,21.86,GTV,UCLA


In [10]:
# reate a connection
conn = sqlite3.connect(os.getcwd()+'/gui/bme_gui/bme_gui/db.sqlite3')
c = conn.cursor()

In [7]:
# # delete all rows
# c.execute('delete from myapp_treatment')
# conn.commit()
# c.execute('select count(*) from myapp_treatment').fetchall()

OperationalError: no such table: myapp_treatment

In [12]:
# conn.close()

In [11]:
# convert data to records
records = list(df.to_records(index=False))
# insert
try:
    c.executemany('insert into myapp_treatment(patient_id, roi_dose, ptv_dose,\
                 ptv_volume, roi_roi_distance, roi_roi_overlap, rt_dose, roi, institution)\
                  values(?,?,?,?,?,?,?,?,?);', records)
    print("successfully instered data")
    print(c.execute('select * from myapp_treatment;').fetchmany(10))
    conn.commit()
except sqlite3.Error as e:
    print(e.args)

('no such table: myapp_treatment',)


In [99]:
c = conn.cursor()
print(c.execute('select count(*) from myapp_treatment limit 10;').fetchmany(10))

[(600,)]


In [100]:
conn.close()

Similarity Table

In [16]:
# convert data to records
records_patientID = list(df[['patient_id']].to_records(index=False))
# print(records_patientID)
# insert
try:
    c.executemany('insert into myapp_similarity(patient_id_id)\
                  values(?);', records_patientID)
    print("successfully instered patient ID")
    print(c.execute('select * from myapp_similarity;').fetchmany(10))
    conn.commit()
except sqlite3.Error as e:
    print(e.args)

successfully instered patient ID
[(1, None, 'UCLA_PR_1'), (2, None, 'UCLA_PR_2'), (3, None, 'UCLA_PR_3'), (4, None, 'UCLA_PR_4'), (5, None, 'UCLA_PR_5'), (6, None, 'UCLA_PR_6'), (7, None, 'UCLA_PR_7'), (8, None, 'UCLA_PR_8'), (9, None, 'UCLA_PR_9'), (10, None, 'UCLA_PR_10')]


In [17]:
print(c.execute('select * from myapp_similarity limit 10;').fetchall())

[(1, None, 'UCLA_PR_1'), (2, None, 'UCLA_PR_2'), (3, None, 'UCLA_PR_3'), (4, None, 'UCLA_PR_4'), (5, None, 'UCLA_PR_5'), (6, None, 'UCLA_PR_6'), (7, None, 'UCLA_PR_7'), (8, None, 'UCLA_PR_8'), (9, None, 'UCLA_PR_9'), (10, None, 'UCLA_PR_10')]


Parsed DICOM table

In [20]:
# delete all rows
c.execute('delete from myapp_parsed_dicom')
conn.commit()
c.execute('select count(*) from myapp_parsed_dicom').fetchall()

[(0,)]

In [21]:
conn.close()

In [24]:
from datetime import datetime

In [25]:
d = datetime.now()
print(d)

2022-04-08 20:40:32.816311


In [29]:
d=d.strftime("%Y-%m-%d %H:%M:%S")
print(d)
print(type(d))

2022-04-08 20:40:32
<class 'str'>


In [28]:
str(d)
print(type(d))

<class 'datetime.datetime'>


### Visualize

dummy data for visualization

In [78]:
similarity = list(np.random.uniform(0.1,0.99,600).round(4))
df['similarity'] = similarity
print(df.head())

  patient_id  roi_dose  ptv_dose  ptv_volume  roi_roi_distance  \
0  UCLA_PR_1     21.30     32.19      189.27             51.43   
1  UCLA_PR_2     23.72     43.32      150.27             35.45   
2  UCLA_PR_3     16.31     34.91      147.15             53.75   
3  UCLA_PR_4     38.00     47.51      141.13             57.38   
4  UCLA_PR_5     29.73     44.85      134.36             51.51   

   roi_roi_overlap  rt_dose         roi institution  similarity  
0             0.75    19.45  Lacrimal R        UCLA      0.6298  
1             0.56    35.28         GTV        UCLA      0.3458  
2             0.15    40.80   Brainstem        UCLA      0.7067  
3             0.76    32.32         GTV        UCLA      0.4354  
4             0.26    39.61      Lens R        UCLA      0.3500  


In [79]:
dff = df[df['similarity']>0.9]

In [87]:
a=None

In [91]:
if a: 
    print("yea")
if not a:
    print("Aa")

Aa
